### test ff
Powered by [Eleonora Priori](https://www.ecoaz.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


The trick above, useful to avoid scrolling output windows, does not work with recent jupyter versions; use settings.

In [1]:
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import pandas as pd
import pickle
import csv
import os
import sys

comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)



#generate random seed
repast4py.random.init(rng_seed=12345)
rng = repast4py.random.default_rng 

In [2]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int): #, labor:int, capital:float, minOrderDuration:int,\
                 #maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 #assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 #sectorialClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        """
        self.labor=labor
        self.capital=capital
        #self.capitalQ= 0
        self.unavailableLabor=0
        self.unavailableCapital=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.sectorialClass=sectorialClass 
        """

        
        


In [3]:

class Model:
    
    def __init__(self):
        
      
        runner.schedule_repeating_event(0.0,  1, self.interactingWithFirms)

        runner.schedule_stop(1)
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################

        fileName = input("file name? ")
        firmCount= int(input("how many firms? "))
        
        #importing csv file containing info about firms 
        with open(fileName, newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',')#, quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1 #to skip the row of the headers
            k=0
            #for each record in .csv
            # case ff_base.csv
            # #,NACE definition,type,dimensional class,Share of firms,Share of firms per sbs sector,sbs reference row

            for row in firmReader:
                #print(row)
                if self.rowNumber>=0:
                    if row[4]=='': row[4]=0
                    for i in range(int(float(row[4]) * firmCount)// rankNum):
                        """labor= rng.integers(row[1], row[2]+1) #+1 because integers exclude extremes 
                        capital= row[3] + rng.random()*(row[4] -row[3])
                        minOrderDuration= row[5]
                        maxOrderDuration= row[6]
                        recipe= row[7] #K/L 
                        laborProductivity= row[8]
                        maxOrderProduction= row[9]
                        avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                        plannedMarkup=row[11]
                        orderObservationFrequency=rng.integers(row[12], row[13]+1)
                        productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                        sectorialClass=int(self.rowNumber)"""
                        aFirm =Firm(k, rank)#, labor, capital, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                #maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                #sectorialClass)
                        context.add(aFirm)
                        k += 1 # first element of the UID of the agents
                self.rowNumber += 1
            self.firmCount=k #one more, here is a count, not an id
        
    #interactingWithFirms
    def interactingWithFirms(self):
        print(len(list(context.agents(agent_type=0))))
        pass

    #finish
    def finish(self):
        print("concluded")
    
    def start(self):
        runner.execute()

In [4]:
def run():

    model = Model() 
    model.start()
    
run()

file name?  ff_base.csv
how many firms?  10


3
3
concluded


In [5]:
#list(context.agents(agent_type=0))

In [6]:
#row